# **Numerical Analysis Project - Week 3**
*Mathilde Bénard, Hector Giraud, Emilie Soret - TEAM 5, GROUP E*

---

In [4]:
import numpy as np
from scipy.integrate import solve_ivp
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

**As explained by our teacher, some of the questions were difficult to answer this week so he asked us to do the best we can and to keep working on the problems this week. As a result, some of the questions might be unfinished but we tried to answer to as many as possible.**

For our team : E.5, we have to consider the following problem 

$$ \begin{cases}
\partial_t u = \beta \partial^2_x u \\
u(t, x=0)=a \\
u(t=0,x)=u_1 cos(\frac{\pi x}{2L}) \\
\partial_x(t,x=L)=0
\end{cases}$$

$$\forall J \in \N, \text{we define } \\
h=\frac{L}{J+1} \\
\forall j \in [1,J], x_j=jh $$

##### **(a). Is it an ODE or a PDE? Is it linear, homogeneous, autonomous?**

This is a **partial differential equation (PDE)** because it involves partial derivatives with respect to both time $t$ and space $x$.
Yes, it is a **linear PDE** because the function $u$ and its derivatives appear only in the first degree and there are no nonlinear terms (such as  $u^2$ or $(\partial_x u)^2$).
The **PDE itself is homogeneous** (since the right-hand side is zero). However, the boundary conditions introduce an inhomogeneity (due to $u(t, x=0) = a$, where  $a > 0$), making the full IBVP **inhomogeneous**.
Yes, the PDE is **autonomous** because the coefficients do not explicitly depend on time  $t$.

 
##### **(b). Implement finite differences in space (forward for the first derivative at L, centered for the second derivative) to define a multidimensional IVP. Show that this method converges.**
Let 
$$J \in \N \\
\text{We define} \\
h_x = \frac{L}{J+1} \\
\forall j \in \llbracket 1, J \rrbracket, x_j = jh \\$$

To implement the scheme we do the following approximations : 

(Forward difference at L): We need to define an extra point for u at x=L+h. We define it s.t. 
 $$\partial_x(t,x=L) \approx \frac{u(t,x=L+h)-u(t,x=L)}{h}=0 \\ \text{which implies that} \\
 u(t, x=L+h)=u(t,x=L)$$ 

 (Centered difference) $$\forall x \in [0,L], \forall t \in [0,T] \\ \partial^2_x u(t,x) \approx \frac{u(t,x+h)+u(t,x-h)-2u(t,x)}{h^2}$$

 This gives us the following discretized problem

 $$ \forall t \in [0,T], \forall j \in \llbracket 1, J+1 \rrbracket \\
    \begin{cases}
    v_0(t)=a \\
    \frac{v_{J+1}(t)-v_J(t)}{h}=0 \\
    v'_j(t)=\beta \frac{v_{j+1}(t)+v_{j-1}-2v_j(t)}{h^2} \\
    v_j(0)=u_1 cos(\frac{\pi x_j}{2L})\\
    \end{cases}$$
    
with 
    $$ (v_j)_{j \in \llbracket 1, J+1 \rrbracket} \in (\R^{\R_+})^{j \in \llbracket 1, J+1 \rrbracket}$$

which can be rewritten in matrix form as 

$$ \begin{cases}
\forall t \in [0,T], V'(t)=\frac{-\beta}{h^2}AV(t)+F \\
V(0)=U_0
\end{cases}$$
with 
$$ V(0)= \begin{bmatrix}
v_1(0) \\
\vdots \\
v_{J+1}(0) \end{bmatrix} \\~\\
V(t)= \begin{bmatrix}
v_1(t) \\
\vdots \\~\\
v_{J+1}(t) \end{bmatrix} \\~\\
V'(t)= \begin{bmatrix}
v'_1(t) \\
\vdots \\
v'_{J+1}(t) \end{bmatrix} \\~\\
A= \begin{bmatrix}
 2 & -1 &  & & 0 \\
 -1 & 2 & -1 &  &  \\
  & \ddots & \ddots & \ddots &  \\
 &  & -1 & 2 & -1\\
0 &  &  & -1 & 1\\ \end{bmatrix} \\~\\
F= \begin{bmatrix}
\frac{\beta a}{h_x^2} \\
0 \\
\vdots \\
0 \end{bmatrix} \\~\\
U_0= \begin{bmatrix}
u^0(x_1) \\
\vdots \\~\\
u^0(x_{J+1}) \end{bmatrix} = \begin{bmatrix}
u_1cos(\frac{\pi x_1}{2L} )\\
\vdots \\~\\
u_1cos(\frac{\pi x_{J+1}}{2L}) \end{bmatrix}\\~\\$$



In [ ]:
# Implementation

J=10
L=1
a=2
def FDS(L,a,u1, J,b, T):
    h=L/(J+1)
    F= np.append(b*a/h**2, np.zeros(J)).reshape(J+1,)
    D=((-1)*np.eye(J+1, k=-1)+2*np.eye(J+1)-1*np.eye(J+1,k=1))
    D[J][J]-=1
    V0=np.array([u1*np.cos((np.pi*j*h)/(2*L)) for j in range(1,J+2)])
    def f(t,X):
       return (-b)/(h**2)*D@X+F
    return solve_ivp(f,(0,T),V0)
sol = FDS(1, 2, 3, 10, 1,10)

**Consistency**

Let $\epsilon (t)$ = $U'(t)$ - $\frac{\beta}{h^2}AU(t) - F = \begin{bmatrix}
\epsilon_1(t) \\
\vdots \\
\epsilon_J(t) \end{bmatrix} $

Let  ${j \in \llbracket 2, J-1 \rrbracket}$

$$\epsilon_j(t) = u'(t)_j - \frac{\beta}{h_x^2}(u_\text{j+n}^t+u_\text{j-n}^t - 2u_j^t)$$

Here the notation " ' " exceptionnally refers to a space derivative.

$$u_\text{j+n}^t = u_j^t + hu'^t_j + \frac{h^2}{2}u''^t_j + \frac{h^3}{6}u'''^t_j + \frac{h^4}{24}u''''^t_{\theta -}$$
$$u_\text{j-n}^t = u_j^t - hu'^t_j + \frac{h^2}{2}u''^t_j - \frac{h^3}{6}u'''^t_j + \frac{h^4}{24}u''''^t_{\theta +}$$

with ${\theta_+},{\theta_-} \in [x_j, x_{j+1}]$. We make the $h^2$ go on the other side.

$$\frac{u_{j+h}^t + u_{j-h}^t - 2u_j^t}{h^2} = u''^t_j + \frac{h^2}{24}(u''''^t_{\theta +} + u''''^t_{\theta -})$$

$$\epsilon_j(t) = \underbrace{\partial_t u_j^t - \beta \partial_x^2 u_j^t}_{=0} - \frac{\beta h^2}{24}(\partial_x^4u^t_{\theta +} + \partial_x^4u^t_{\theta -})$$

$$\lvert \epsilon_j(t) \rvert \leq \frac{\beta h_x^2}{12}\underbrace{\lVert \delta_x^4 u(t,\cdot)\rVert _\text{$\infty$, [0,L]}}_{\in \mathbb{R}_+}$$

as $u(t,\cdot)$ is continuous on a segment.

$$\lVert \epsilon_j(t) \rVert \leq  \frac{\beta h_x^2}{12}\lVert \delta_x^4 u(t,\cdot)\rVert _\text{$\infty$, [0,L]}$$

**Stability**

1- Let us show that 
$E'(t)= \epsilon(t) - \frac{\beta}{h_x^2}AE(t)$(1)

$$\begin {split}
 E'(t) &  = U'(t)-V'(t) \\
   &  = \epsilon(t) - \frac{\beta}{h_x^2}AU(t)+F +\frac{\beta}{h_x^2}AV(t)-F \\
& = \epsilon(t) - \frac{\beta}{h_x^2}A (U(t)-V(t)) \\
 &    = \epsilon(t) - \frac{\beta}{h_x^2}A E(t) \\
    \end{split}$$
    
2- Let us solve (1) using the varying constant method: \
We look for the function $ E \in (\R^{J+1})^{\R_+}$ in the shape $E(t)=e^{-\frac{\beta}{h_x^2}At}Y(t)$ with $Y \in (\R^{J+1})^{\R_+}$ differentiable. 
(1) rewrites 

$$\begin{split}
E'(t) &=  \epsilon(t) - \frac{\beta}{h_x^2}AE(t) \\
-\frac{\beta}{h_x^2}A e^{-\frac{\beta}{h_x^2}At}Y(t)+ Y'(t)e^{-\frac{\beta}{h_x^2}At} & = \epsilon(t) - \frac{\beta}{h_x^2}AE(t)\\
Y'(t)e^{-\frac{\beta}{h_x^2}At} & = \epsilon(t) \\
Y'(t) & = e^{\frac{\beta}{h_x^2}At}\epsilon(t)
\end{split}$$

Solving for Y we get 

$$\begin{split}
Y'(t) & = e^{\frac{\beta}{h_x^2}At}\epsilon(t) \\
\int_0^t Y'(s) ds & = \int_0^t e^{\frac{\beta}{h_x^2}As}\epsilon(s) ds \\
Y(t)-Y(0) & = \int_0^t e^{\frac{\beta}{h_x^2}As}\epsilon(s) ds \\
\end{split}$$
Since $Y(0)=E(0)=0$, we finally have
$Y(t) = \int_0^t e^{\frac{\beta}{h_x^2}As}\epsilon(s) ds$

Hence $\forall t \in [0,T], E(t)= \int_0^t e^{\frac{\beta}{h_x^2}A(t-s)}\epsilon(s) ds$ (2)

3- From (2), $\forall t \in [0,T]$
$$\begin{split}
||E(t)||_2& = ||\int_0^t e^{\frac{\beta}{h_x^2}A(t-s)}\epsilon(s) ds||_2\\
||E(t)||_2& \leqslant \int_0^t ||e^{\frac{\beta}{h_x^2}A(t-s)}\epsilon(s)||_2 ds \\
||E(t)||_2& \leqslant \int_0^t |||e^{\frac{\beta}{h_x^2}A(t-s)}|||_2||\epsilon(s)||_2 ds (3)\\
\end{split}$$

In exercise 11, we showed that $\forall S \in Sym_{J}(\R), e^{-\alpha St}  \in Sym_{J}(\R)$ for some $\alpha \in \R$
Hence, as $A \in Sym_{J}(\R),  e^{\frac{\beta}{h_x^2}A(t-s)} \in Sym_{J}(\R)$ and $|||e^{\frac{\beta}{h_x^2}A(t-s)}|||_2= max\{sp(e^{\frac{\beta}{h_x^2}A(t-s)})\}$\
Let $\lambda \in sp(A)$ s.t. $|\lambda|= max\{sp(A)\}$. \
We also showed that $e^{\frac{\beta}{h_x^2}|\lambda|(t-s)}=max\{sp(e^{\frac{\beta}{h_x^2}A(t-s)})\}$. Hence (3) rewrites 
$$\begin{split}
||E(t)||_2& \leqslant \int_0^t e^{\frac{\beta}{h_x^2}|\lambda|(t-s)} ||\epsilon(s)||_2 ds \\
||E(t)||_2& \leqslant \int_0^t e^{\frac{\beta}{h_x^2}|\lambda|(t-s)} \underset{s\in[0,T]}{sup}||\epsilon(s)||_2 ds \\
||E(t)||_2& \leqslant  \underset{s\in[0,T]}{sup}||\epsilon(s)||_2 \int_0^t e^{\frac{\beta}{h_x^2}|\lambda|(t-s)} ds \\
\end{split}$$
$\forall t \in [0,T], \forall s \in [0,t], \frac{\beta}{h_x^2}|\lambda|(t-s) \geqslant 0$ \
Hence $\int_0^t e^{\frac{\beta}{h_x^2}|\lambda|(t-s)} ds \leqslant \int_0^t 1 ds \leqslant t \leqslant T$ and 
$$\begin{split}
||E(t)||_2& \leqslant  \underset{s\in[0,T]}{sup}||\epsilon(s)||_2 T \\
||E(t)||_2& \leqslant  CTh_x^2 
\end{split}$$

Hence as $||E(t)||_2 \leqslant  CTh_x^2 \underset{J \to \infty}{\longrightarrow} 0$, the scheme is convergent with respect to the infinite norm. 

c) Let $N \in \N$ and $T \in \R^+$ To implement Euler's forward scheme in time we define \
$h_t=\frac{T}{N+1} \\
\forall n \in \llbracket 1,N \rrbracket, t^n = nh_t $ \
We approximate $ \forall n \in \llbracket 1,N \rrbracket, V'(t^n) \approx \frac{V(t^{n+1})-V(t^n)}{h}$. \
From now on, $\forall n \in \llbracket 1,N \rrbracket$, we denote $V(t^{n})=V^n$\
\
We obtain the following discretized problem \
$\begin{cases} 
V^0=U^0 \\
\forall n \in \llbracket 1,N \rrbracket, V^{n+1}=(\frac{-h_t \beta}{h_x^2}A + I_{J+1})V^n +F 
\end{cases}$


**Consistency**

The **consistency error** is defined as:

$$
\mathcal{E}^n = U^{n+1}-(\frac{-h_t \beta}{h_x^2}A + I_{J+1})U^n +F $$

From the **Taylor-Lagrange theorem**, for all $n \in [1, N+1]$, let $\theta_1^n, ..., \theta_J^n \in [t^{n-1}, t^n]$ such that:

$$
U^{n+1} = U^{n} + h_t \begin{bmatrix} \partial_t u (t^{n}, x_1) \\ \vdots \\ \partial_t u (t^{n}, x_J) \end{bmatrix} + \frac{h_t^2}{2} \begin{bmatrix} \partial_t^2 u (\theta_1^n, x_1) \\ \vdots \\ \partial_t^2 u (\theta_J^n, x_J) \end{bmatrix} = U^{n} + h_t \partial_t U^{n} + \frac{h_t^2}{2}U(\theta^n).
$$

From the IVP $\forall n \in \llbracket 1, N \rrbracket, \partial _t U^n = U^{n'}=\frac{-\beta}{h^2}AU^n+F $. Thus

$$ \begin{split}
U^{n+1} & = U^{n} + h_t \frac{-\beta}{h^2}AU^n+F + \frac{h_t^2}{2}U(\theta^n) \\
U^{n+1} - U^{n} - h_t \frac{-\beta}{h^2}AU^n+F & = \frac{h_t^2}{2}U(\theta^n) \\
\mathcal{E}^n & =\frac{h_t^2}{2}U(\theta^n)\\~\\
||\mathcal{E}^n||_{\infty} & =||\frac{h_t^2}{2}U(\theta^n)||_{\infty} = \frac{h_t^2}{2} \underset{j \in \llbracket 1, J+1  \rrbracket}{sup} |\partial_t^2 u (\theta_j^n, x_j)|\\
\underset{n}{max}||\mathcal{E}^n||_{\infty} & \leqslant \underset{n}{max} (\frac{h_t^2}{2} \underset{j \in \llbracket 1, J+1  \rrbracket}{sup} |\partial_t^2 u (\theta_j^n, x_j)| )\\
 & \underset{J \to \infty}{\to}0
\end{split}$$
Hence, the scheme is consistent with respect to the infinite norm

**Stability**
Drawing on exercise 11, to show stability, we will show that the following propositons are equivalent 


(i) $\exists K \geqslant 0, \forall J \in \N, \forall N \in \N, \forall (z^n)_{0 \leqslant n \leqslant N+1}, (w^n)_{0 \leqslant n \leqslant N+1}, (\eta^n)_{0 \leqslant n \leqslant N+1} \in (\R^{J+1})^{N+2}: $ \
$\begin{cases} 
w^0=z^0+h_t \eta^0 \\
\forall n \in \llbracket 1, N \rrbracket, z^{n+1}=(\frac{-h_t \beta}{h_x^2}+I_{J+1})z^n +F \\
\forall n \in \llbracket 1, N \rrbracket, w^{n+1}=(\frac{-h_t \beta}{h_x^2}+I_{J+1})w^n +F + h_t\eta^{n+1} \\
\end{cases}$ 

$\implies \underset{0 \leqslant n \leqslant N+1}{max}||z^n -w^n||_2 \leqslant \underset{0 \leqslant n \leqslant N+1}{max} ||\eta^n||_{\infty}$

(ii) $\forall J \in \N, |||(\frac{-h_t \beta}{h_x^2}A+I_{J+1})|||_2 = |||\Beta|||_2 \leqslant 1$

(iii) tbd

(iv) tbd

**(ii) $\implies$ (i)**

Suppose (ii)
Let $n \in \llbracket 1, N \rrbracket$,

$$\begin{split}
w^{n}-z^{n} & =\Beta (w^{n-1}-z^{n-1})+h_t\eta^{n} \\

 & = \Beta (\Beta (w^{n-2}-z^{n-2})+h_t\eta^{n-1})+h_t\eta^{n} \\
 & = \Beta^{n}(w^0-z^0) +h_t \sum \limits^{n}_{k=0}(\Beta^{n-k}\eta^k)  \\
 \end {split}$$ 
 
 
 by induction 
 Since $w^0-z^0=0$, we obtain, \
$\forall n \in \llbracket 1, N \rrbracket, w^{n}-z^{n}=h_t \sum \limits^{n}_{k=0}\Beta^{n-k}\eta^k \\$ 

$$\begin{split}
||w^{n}-z^{n}||_2 & =||h_t \sum \limits^{n}_{k=0}\Beta^{n-k}\eta^k||_2\\
& \leqslant h_t \sum \limits^{n}_{k=0}||\Beta^{n-k}\eta^k||_2\\
& \leqslant h_t \sum \limits^{n}_{k=0}|||\Beta^{n-k}|||_2||\eta^k||_2\\
& \leqslant h_t \sum \limits^{n}_{k=0}1||\eta^k||_2\\
& \leqslant h_t \sum \limits^{n}_{k=0}1 \underset{0 \leqslant n \leqslant N+1}{max}||\eta^n||_2\\
& \leqslant h_t (n+1) \underset{0 \leqslant n \leqslant N+1}{max}||\eta^n||_2\\
\underset{0 \leqslant n \leqslant N+1}{max}||w^{n}-z^{n}||_2 &  \leqslant h_t (N+2) \underset{0 \leqslant n \leqslant N+1}{max}||\eta^n||_2\\
&  \leqslant 2T \underset{0 \leqslant n \leqslant N+1}{max}||\eta^n||_2\\
\end{split}$$
Hence (ii) $\implies$ (i)

**(i) $\implies$ (ii)**

Let us proceed by contraposition and suppose that $|||\Beta||| = max\{|sp(\Beta)|\} \geqslant 1$ 
Let $\lambda = max\{|sp(\Beta)|\}$ and $z^0$ the eigenvector associated with $\lambda$
Define the sequence $(z_n)_{0 \leqslant n \leqslant N+1}$ s.t. $\forall n \in \llbracket 0, N \rrbracket, z^{n+1}=Bz^n i.e. z^n=\Beta^n z^0 = \lambda^n z^0$ by induction and the definition of $z^0$
Let $w=(0, \cdots, 0)$ and $\eta = (-z^0, 0, \cdots, 0) \in \R^{N+1}$
Those sequences verify (i). Moreover, $\forall n \in \llbracket 0, N \rrbracket$, 

$||z^n-w^n||_2 = ||z^n||_2 = |\lambda|^n ||z^0||_2 $


As $|\lambda| \geqslant 1 $ by hypothesis, 

$\underset{0 \leqslant n \leqslant N+1}{max}||z^n-w^n||_2 = ||z^n||_2 = |\lambda|^{N+1} ||z^0||_2 
\leqslant \underset{0 \leqslant n \leqslant N+1}{max}||\eta^n||_{\infty}=||z^0||_{\infty} \\
\underset{N \to \infty}{lim} (\underset{0 \leqslant n \leqslant N+1}{max}||z^n-w^n||_2) = \infty \geqslant \underset{N \to \infty}{lim}||z^0||_{\infty} = ||z^0||_{\infty}  $

This is a contradiction. Hence (i) $\implies $(ii)


In [9]:
#Implementation

def euler_forward(T,beta, L,J, N, u1):
    h_t=T/(N+1)
    h_x=L/(J+1)
    
    I = np.zeros(J+1)
    A = 2*np.eye(J+1)-1*np.eye(J+1, k=-1)-1*np.eye(J+1, k=1)
    A[J][J]-=1
    B = (-h_t*beta/h_x**2)*A+I
    F= np.append(beta*a/h_x**2, np.zeros(J)).reshape(J+1,)
    
    V0=np.array([u1*np.cos((np.pi*j*h_x)/(2*L)) for j in range(1,J+2)])
    V=[V0]
    t=[0]
    
    for n in range(1, N+1):
        Vn=V[-1]
        V.append(np.dot(B,Vn)+F)
        t.append(n*h_t)
    return V,t